<a href="https://colab.research.google.com/github/sujithkumarmp/google-colab/blob/main/gen-ai/ngram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:

import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from itertools import islice

# Sample text data
text = "We are learning PyTorch for building N-gram models. PyTorch is fun and powerful."

# Preprocessing: Tokenize and create N-grams
def generate_ngrams(tokens, n):
    return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

tokens = text.lower().split()
n = 3  # Trigram example
ngrams = generate_ngrams(tokens, n)

# Vocabulary and mapping
vocab = set(tokens)
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# Prepare data
data = [(torch.tensor([word_to_idx[w] for w in ngram[:-1]], dtype=torch.long),
         torch.tensor(word_to_idx[ngram[-1]], dtype=torch.long)) for ngram in ngrams]

# Define the model
class NGramLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view(1, -1)
        out = torch.relu(self.linear1(embeds))
        out = self.linear2(out)
        return out

# Hyperparameters
embedding_dim = 10
context_size = n - 1
model = NGramLanguageModel(len(vocab), embedding_dim, context_size)

# Loss and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(100):
    total_loss = 0
    for context, target in data:
        optimizer.zero_grad()
        output = model(context)
        loss = loss_function(output, target.view(1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss:.4f}")

# Testing the model
context = torch.tensor([word_to_idx["pytorch"], word_to_idx["is"]], dtype=torch.long)
predicted = model(context).argmax(dim=1).item()
print(f"Prediction for context 'pytorch is': {idx_to_word[predicted]}")


Epoch 0, Loss: 26.7399
Epoch 10, Loss: 11.6344
Epoch 20, Loss: 4.7709
Epoch 30, Loss: 2.3051
Epoch 40, Loss: 1.3659
Epoch 50, Loss: 0.9268
Epoch 60, Loss: 0.6849
Epoch 70, Loss: 0.5353
Epoch 80, Loss: 0.4354
Epoch 90, Loss: 0.3645
Prediction for context 'pytorch is': fun
